In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv("D:/Project 2/New_naukri.csv")
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
df.head()

,ROLES,Companies,Experience,Salary,Location,Description,Skills,Posted_Date,Present_Date,TYPE,Title_new
0,data scientist/analyst - machine learning/deep...,EchoIndia,3-6,NaN,Delhi,- Strong in statistical models and machine le...,"python,machine learning,tableau,power bi,arcgi...",9 DAYS AGO,2020-10-24,HOT JOB,Scientist
1,"data scientist, machine learning",Fractal Analytics Ltd,3-7,NaN,Bengaluru,of on end-to-end data science pipeline: prob...,"upply chain management,linux,rf,rdbms,bfsi,con...",9 DAYS AGO,2020-10-24,HOT JOB,Scientist
2,"data scientist, i",iXplore Technologies,0-2,NaN,Bengaluru,"Bachelors degree, Masters or PhD in statistics...","oftware development,machine learning,big data,...",9 DAYS AGO,2020-10-24,HOT JOB,Scientist
3,explore job openings on data scientist!!,Bristlecone India Limited,4-9,300000 - 800000,India,s-on python programm with design data science...,"python,data science,machine learning,cloud,aws...",7 DAYS AGO,2020-10-24,HOT JOB,Scientist
4,inviting application for data scientist - mana...,GENPACT India Private Limited,6-11,NaN,Bengaluru,"With a startup spirit and 90, 000+ curious and...","machine learning,statistics,deep learning,reco...",10 DAYS AGO,2020-10-24,PREMIUM,Manager


In [3]:
df['Search']=df['ROLES']+","+df['Location']

In [4]:
df['Search'][0]

'data scientist/analyst - machine learning/deep learning,Delhi'

In [5]:
df['Search'].value_counts()

data scientist,Bengaluru                                  87
data engineer,Bengaluru                                   59
data analyst,Bengaluru                                    45
data scientist,Delhi NCR                                  43
senior data scientist,Bengaluru                           36
                                                          ..
senior data scientist - internet/online,Bengaluru          1
python developer - orm/ javascript,Bengaluru               1
software engineer c/c /python/perl,Chennai, dharmapuri     1
data scientist (full time),Bengaluru                       1
sr. associate - projects,Gurgaon Gurugram                  1
Name: Search, Length: 4606, dtype: int64

In [6]:
df['Source']=df['ROLES']+' '+df['Location']
df['Source'][0]

'data scientist/analyst - machine learning/deep learning Delhi'

In [7]:
#df=df.drop_duplicates(subset='ROLES',keep='first').reset_index(drop=True)
#df.shape

In [8]:
import re
import nltk
from nltk.corpus import stopwords
#from nltk.corpus import words
from nltk.stem import WordNetLemmatizer 

In [9]:
lema=WordNetLemmatizer()
title=[]
for i in range(len(df['Source'])):
    tit=re.sub('[^a-zA-Z]',' ',df['Source'][i])
    tit=tit.lower()
    tit = tit.split()
    tit=[lema.lemmatize(word) for word in tit if not word in set(stopwords.words('english'))]
    tit=" ".join(tit)
    title.append(tit)

In [10]:
df['Clean_data']=title
df['Clean_data']=df['Clean_data'].fillna('')

In [11]:
df['Clean_data'].value_counts()

data scientist bengaluru                                      91
data engineer bengaluru                                       60
data analyst bengaluru                                        46
data scientist delhi ncr                                      43
data scientist india                                          36
                                                              ..
python developer experience data analysis kochi                1
job opportunity sr adobe target consultant bengaluru           1
lead engineer data platform india                              1
data scientist data analyst data engineering chennai india     1
lte design engineer bengaluru                                  1
Name: Clean_data, Length: 4442, dtype: int64

In [12]:
df=df.drop_duplicates(subset='Search',keep='first').reset_index(drop=True)
df.shape

(4606, 14)

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfv=TfidfVectorizer(min_df=3,max_features=None,ngram_range=(1,3))

In [14]:
tfv_matrix=tfv.fit_transform(df['Clean_data'])

In [15]:
tfv_matrix

<4606x2698 sparse matrix of type '<class 'numpy.float64'>'
	with 42769 stored elements in Compressed Sparse Row format>

In [16]:
from sklearn.metrics.pairwise import sigmoid_kernel
sig = sigmoid_kernel(tfv_matrix,tfv_matrix)

In [17]:
sig[0]

array([0.76174977, 0.7616242 , 0.76160907, ..., 0.76159416, 0.7615979 ,
       0.76159727])

In [18]:
indices=pd.Series(df.index,index=df['Search']).drop_duplicates()
indices

Search
data scientist/analyst - machine learning/deep learning,Delhi                       0
data scientist, machine learning,Bengaluru                                          1
data scientist, i,Bengaluru                                                         2
explore job openings on data scientist!!,India                                      3
inviting application for data scientist - manager/sr. manager - blr,Bengaluru       4
                                                                                 ... 
ai/ml - testing lead / manager on initial wfh basis,Mysuru                       4601
full stack developer,Delhi                                                       4602
artificial intelligence,Jaipur                                                   4603
lead technical architect,Delhi NCR                                               4604
analyst with leading group of company,Chennai                                    4605
Length: 4606, dtype: int64

In [19]:
def rec_tit(title,sig=sig):
    idx=indices[title]
    
    sig_scores=list(enumerate(sig[idx]))
    #sortbased on sigmoid values
    sig_scores=sorted(sig_scores,key=lambda x :x[1],reverse=True)
    
    sig_scores=sig_scores[1:11]
    
    title_indices=[i[0] for i in sig_scores]
    
    role=df['Search'].iloc[title_indices]
    comp=df['Companies'].iloc[title_indices]
    loc=df['Location'].iloc[title_indices]
    sal=df['Salary'].iloc[title_indices]
    skills=df['Skills'].iloc[title_indices]
    job_titles=df['Title_new'].iloc[title_indices]
    sig_score=sig_scores
    # input values into dataframe
    rec_df=pd.DataFrame({'Given Job_roles':role,'Company':comp,'location':loc,'salary':sal,'Skills':skills,'Titles_of_job':job_titles})
    #resultant dataframe is again sorts based on custom_score
    #rec_df=rec_df.sort_values("Scores", ascending=False)
    return rec_df

In [20]:
df.Search.value_counts()

member technical staff - qa [cloud],Bengaluru                                    1
database developer - mumbai - senior analyst /manager,Mumbai                     1
senior analyst _ cost analytics,Bengaluru                                        1
ios developer,Hyderabad                                                          1
tracxn - lead - full stack - node/ express/ react/ mongo (4-10 yrs),Bengaluru    1
                                                                                ..
python developer,Visakhapatnam                                                   1
pifs_tsdc_consultant_ba_senior,Bengaluru, Bangalore                              1
developer,india                                                                  1
quantitative researcher,Bengaluru                                                1
sr. associate - projects,Gurgaon Gurugram                                        1
Name: Search, Length: 4606, dtype: int64

In [21]:
rec_tit("tsql expert & data analyst,Hyderabad")

,Given Job_roles,Company,location,salary,Skills,Titles_of_job
612,"data analyst,Hyderabad",IVY SOFTWARE DEVELOPMENT SERVICES PVT LTD,Hyderabad,NaN,"python,machine learning,tableau,power bi,r,dat...",Analyst
615,"data analyst i,Hyderabad",DUN & BRADSTREET INFORMATION SERVICES INDIA PR...,Hyderabad,NaN,"data analysis,metadata,data structures,dnb,dat...",Analyst
822,"ediscovery data analyst,Hyderabad",Law In Order Pty Ltd,Hyderabad,NaN,"data conversion,document review,project manage...",Analyst
569,"sr . reference data analyst,Hyderabad",Vertafore,Hyderabad,NaN,"raining,qa,product innovation,manager technolo...",Analyst
485,"intern - data analyst,Hyderabad",Leva Data Software Solutions Pvt. Ltd,Hyderabad,200000 - 250000,"python,python",Analyst
441,"trainee data analyst,Hyderabad",Dynata,Hyderabad,NaN,"java,platforms,c++,survey,c,xml,programming,te...",Analyst
2398,"data analyst_hyderabad,Hyderabad",Arise Global Services Private Limited,Hyderabad,800000 - 1200000,"hive,data analytics,data management,digital ma...",Analyst
829,"creative data analyst,Hyderabad",Synergiz Global Services Private Limited,Hyderabad,NaN,"civil construction,interpersonal skills,tablea...",Analyst
465,"senior data analyst,Hyderabad",Smart Minds,Hyderabad,NaN,"python,consumer electronics,r,mysql,data analy...",Lead
574,"senior data analysts,Hyderabad",Pratibha Analytics,Hyderabad,NaN,"echnical product configuration,c++,c,design,my...",Lead


In [22]:
df.head(2)

,ROLES,Companies,Experience,Salary,Location,Description,Skills,Posted_Date,Present_Date,TYPE,Title_new,Search,Source,Clean_data
0,data scientist/analyst - machine learning/deep...,EchoIndia,3-6,NaN,Delhi,- Strong in statistical models and machine le...,"python,machine learning,tableau,power bi,arcgi...",9 DAYS AGO,2020-10-24,HOT JOB,Scientist,data scientist/analyst - machine learning/deep...,data scientist/analyst - machine learning/deep...,data scientist analyst machine learning deep l...
1,"data scientist, machine learning",Fractal Analytics Ltd,3-7,NaN,Bengaluru,of on end-to-end data science pipeline: prob...,"upply chain management,linux,rf,rdbms,bfsi,con...",9 DAYS AGO,2020-10-24,HOT JOB,Scientist,"data scientist, machine learning,Bengaluru","data scientist, machine learning Bengaluru",data scientist machine learning bengaluru


In [23]:
df_new=df.copy()
df_new

,ROLES,Companies,Experience,Salary,Location,Description,Skills,Posted_Date,Present_Date,TYPE,Title_new,Search,Source,Clean_data
0,data scientist/analyst - machine learning/deep...,EchoIndia,3-6,NaN,Delhi,- Strong in statistical models and machine le...,"python,machine learning,tableau,power bi,arcgi...",9 DAYS AGO,2020-10-24,HOT JOB,Scientist,data scientist/analyst - machine learning/deep...,data scientist/analyst - machine learning/deep...,data scientist analyst machine learning deep l...
1,"data scientist, machine learning",Fractal Analytics Ltd,3-7,NaN,Bengaluru,of on end-to-end data science pipeline: prob...,"upply chain management,linux,rf,rdbms,bfsi,con...",9 DAYS AGO,2020-10-24,HOT JOB,Scientist,"data scientist, machine learning,Bengaluru","data scientist, machine learning Bengaluru",data scientist machine learning bengaluru
2,"data scientist, i",iXplore Technologies,0-2,NaN,Bengaluru,"Bachelors degree, Masters or PhD in statistics...","oftware development,machine learning,big data,...",9 DAYS AGO,2020-10-24,HOT JOB,Scientist,"data scientist, i,Bengaluru","data scientist, i Bengaluru",data scientist bengaluru
3,explore job openings on data scientist!!,Bristlecone India Limited,4-9,300000 - 800000,India,s-on python programm with design data science...,"python,data science,machine learning,cloud,aws...",7 DAYS AGO,2020-10-24,HOT JOB,Scientist,"explore job openings on data scientist!!,India",explore job openings on data scientist!! India,explore job opening data scientist india
4,inviting application for data scientist - mana...,GENPACT India Private Limited,6-11,NaN,Bengaluru,"With a startup spirit and 90, 000+ curious and...","machine learning,statistics,deep learning,reco...",10 DAYS AGO,2020-10-24,PREMIUM,Manager,inviting application for data scientist - mana...,inviting application for data scientist - mana...,inviting application data scientist manager sr...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4601,ai/ml - testing lead / manager on initial wfh ...,ArrisTechnologies,8-12,2000000 - 3000000,Mysuru,and Opportunity:We are currently for AI / ML...,"esting,machine learning,project management,pro...",30+ DAYS AGO,2020-10-24,HOT JOB,Lead,ai/ml - testing lead / manager on initial wfh ...,ai/ml - testing lead / manager on initial wfh ...,ai ml testing lead manager initial wfh basis m...
4602,full stack developer,Glowfin India Solutions,6-8,NaN,Delhi,Lead Python Developer be an Expert programmer...,"python,machine learning,artificial intelligenc...",30+ DAYS AGO,2020-10-24,PREMIUM,Developer,"full stack developer,Delhi",full stack developer Delhi,full stack developer delhi
4603,artificial intelligence,Sdora Consulting Pvt Ltd,2-4,400000 - 750000,Jaipur,"Proven with deep learn, Tensorflow","python,machine learning,artificial intelligenc...",28 DAYS AGO,2020-10-24,HOT JOB,Generic,"artificial intelligence,Jaipur",artificial intelligence Jaipur,artificial intelligence jaipur
4604,lead technical architect,STRATEGIC TALENT PARTNER LLP,7-12,NaN,Delhi NCR,"Key Skills : Technical architect with java, an...","java,machine learning,mobile application devel...",22 DAYS AGO,2020-10-24,HOT JOB,Lead,"lead technical architect,Delhi NCR",lead technical architect Delhi NCR,lead technical architect delhi ncr


In [24]:
df

,ROLES,Companies,Experience,Salary,Location,Description,Skills,Posted_Date,Present_Date,TYPE,Title_new,Search,Source,Clean_data
0,data scientist/analyst - machine learning/deep...,EchoIndia,3-6,NaN,Delhi,- Strong in statistical models and machine le...,"python,machine learning,tableau,power bi,arcgi...",9 DAYS AGO,2020-10-24,HOT JOB,Scientist,data scientist/analyst - machine learning/deep...,data scientist/analyst - machine learning/deep...,data scientist analyst machine learning deep l...
1,"data scientist, machine learning",Fractal Analytics Ltd,3-7,NaN,Bengaluru,of on end-to-end data science pipeline: prob...,"upply chain management,linux,rf,rdbms,bfsi,con...",9 DAYS AGO,2020-10-24,HOT JOB,Scientist,"data scientist, machine learning,Bengaluru","data scientist, machine learning Bengaluru",data scientist machine learning bengaluru
2,"data scientist, i",iXplore Technologies,0-2,NaN,Bengaluru,"Bachelors degree, Masters or PhD in statistics...","oftware development,machine learning,big data,...",9 DAYS AGO,2020-10-24,HOT JOB,Scientist,"data scientist, i,Bengaluru","data scientist, i Bengaluru",data scientist bengaluru
3,explore job openings on data scientist!!,Bristlecone India Limited,4-9,300000 - 800000,India,s-on python programm with design data science...,"python,data science,machine learning,cloud,aws...",7 DAYS AGO,2020-10-24,HOT JOB,Scientist,"explore job openings on data scientist!!,India",explore job openings on data scientist!! India,explore job opening data scientist india
4,inviting application for data scientist - mana...,GENPACT India Private Limited,6-11,NaN,Bengaluru,"With a startup spirit and 90, 000+ curious and...","machine learning,statistics,deep learning,reco...",10 DAYS AGO,2020-10-24,PREMIUM,Manager,inviting application for data scientist - mana...,inviting application for data scientist - mana...,inviting application data scientist manager sr...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4601,ai/ml - testing lead / manager on initial wfh ...,ArrisTechnologies,8-12,2000000 - 3000000,Mysuru,and Opportunity:We are currently for AI / ML...,"esting,machine learning,project management,pro...",30+ DAYS AGO,2020-10-24,HOT JOB,Lead,ai/ml - testing lead / manager on initial wfh ...,ai/ml - testing lead / manager on initial wfh ...,ai ml testing lead manager initial wfh basis m...
4602,full stack developer,Glowfin India Solutions,6-8,NaN,Delhi,Lead Python Developer be an Expert programmer...,"python,machine learning,artificial intelligenc...",30+ DAYS AGO,2020-10-24,PREMIUM,Developer,"full stack developer,Delhi",full stack developer Delhi,full stack developer delhi
4603,artificial intelligence,Sdora Consulting Pvt Ltd,2-4,400000 - 750000,Jaipur,"Proven with deep learn, Tensorflow","python,machine learning,artificial intelligenc...",28 DAYS AGO,2020-10-24,HOT JOB,Generic,"artificial intelligence,Jaipur",artificial intelligence Jaipur,artificial intelligence jaipur
4604,lead technical architect,STRATEGIC TALENT PARTNER LLP,7-12,NaN,Delhi NCR,"Key Skills : Technical architect with java, an...","java,machine learning,mobile application devel...",22 DAYS AGO,2020-10-24,HOT JOB,Lead,"lead technical architect,Delhi NCR",lead technical architect Delhi NCR,lead technical architect delhi ncr


In [25]:
df_new=df_new[['ROLES','Companies','Experience','Salary','Location','Skills','Search','Clean_data','Title_new']]
df_new

,ROLES,Companies,Experience,Salary,Location,Skills,Search,Clean_data,Title_new
0,data scientist/analyst - machine learning/deep...,EchoIndia,3-6,NaN,Delhi,"python,machine learning,tableau,power bi,arcgi...",data scientist/analyst - machine learning/deep...,data scientist analyst machine learning deep l...,Scientist
1,"data scientist, machine learning",Fractal Analytics Ltd,3-7,NaN,Bengaluru,"upply chain management,linux,rf,rdbms,bfsi,con...","data scientist, machine learning,Bengaluru",data scientist machine learning bengaluru,Scientist
2,"data scientist, i",iXplore Technologies,0-2,NaN,Bengaluru,"oftware development,machine learning,big data,...","data scientist, i,Bengaluru",data scientist bengaluru,Scientist
3,explore job openings on data scientist!!,Bristlecone India Limited,4-9,300000 - 800000,India,"python,data science,machine learning,cloud,aws...","explore job openings on data scientist!!,India",explore job opening data scientist india,Scientist
4,inviting application for data scientist - mana...,GENPACT India Private Limited,6-11,NaN,Bengaluru,"machine learning,statistics,deep learning,reco...",inviting application for data scientist - mana...,inviting application data scientist manager sr...,Manager
...,...,...,...,...,...,...,...,...,...
4601,ai/ml - testing lead / manager on initial wfh ...,ArrisTechnologies,8-12,2000000 - 3000000,Mysuru,"esting,machine learning,project management,pro...",ai/ml - testing lead / manager on initial wfh ...,ai ml testing lead manager initial wfh basis m...,Lead
4602,full stack developer,Glowfin India Solutions,6-8,NaN,Delhi,"python,machine learning,artificial intelligenc...","full stack developer,Delhi",full stack developer delhi,Developer
4603,artificial intelligence,Sdora Consulting Pvt Ltd,2-4,400000 - 750000,Jaipur,"python,machine learning,artificial intelligenc...","artificial intelligence,Jaipur",artificial intelligence jaipur,Generic
4604,lead technical architect,STRATEGIC TALENT PARTNER LLP,7-12,NaN,Delhi NCR,"java,machine learning,mobile application devel...","lead technical architect,Delhi NCR",lead technical architect delhi ncr,Lead


In [26]:
#df_new.to_csv('D:/Project 2/naukri_recommend.csv')

In [30]:
df_new['Search']=df_new['Search'].str.lower()

C:\Users\vinay\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [31]:
df_new['Search'].head(2)

0    data scientist/analyst - machine learning/deep...
1           data scientist, machine learning,bengaluru
Name: Search, dtype: object